In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [18]:
# 卷积操作
def corr2d(X, K):
    h, w = K.shape
    U = torch.zeros(X.shape[0] - h + 1, X.shape[1] - w + 1)
    for i in range(U.shape[0]):
        for j in range(U.shape[1]):
            U[i, j]  = (X[i:i+h, j:j+w] * K).sum()
    return U

In [13]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])

In [19]:
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [20]:
torch.rand(2)

tensor([0.0438, 0.8883])

In [22]:
# 卷积层实现
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    
    def forward(self, X):
        return corr2d(X, self.weight) + self.bias

In [23]:
# 边缘目标检测
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [29]:
K = torch.tensor([1, -1]).reshape(1, -1)

In [34]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [35]:
Y.shape

torch.Size([6, 7])

In [33]:
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [38]:
# 使用torch自带卷机模块实现
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 参数：
# 批量大小、通道、高度、宽度
X = X.reshape(1, 1, 6, 8)
Y = Y.reshape(1, 1, 6, 7)

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y - Y_hat) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    print('epoch:{} loss:{}'.format(i, l.sum()))

epoch:0 loss:16.620126724243164
epoch:1 loss:7.261341094970703
epoch:2 loss:3.2646377086639404
epoch:3 loss:1.5230464935302734
epoch:4 loss:0.7427846193313599
epoch:5 loss:0.38036927580833435
epoch:6 loss:0.20451897382736206
epoch:7 loss:0.11495161056518555
epoch:8 loss:0.06703978031873703
epoch:9 loss:0.040231067687273026


In [39]:
conv2d.weight.data[:]

tensor([[[[ 0.9678, -1.0067]]]])

In [40]:
X.shape

torch.Size([1, 1, 6, 8])

In [41]:
(1, 1) + (6, 8)

(1, 1, 6, 8)

In [47]:
# 为了方便起见，我们定义了一个计算卷积层的函数。
# 此函数初始化卷积层权重，并对输入和输出提高和缩减相应的维数
# padding操作
def comp_conv2d(conv2d, X):
    X = X.reshape((1, 1) + X.shape)
    Y = conv2d(X)
    
    return Y.reshape(Y.shape[2:])

In [46]:
X.reshape(X.shape[2:])

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [45]:
X.shape[2:]

torch.Size([6, 8])

In [48]:
# 请注意，这里每边都填充了1行或1列，因此总共添加了2行或2列
# padding指其实是1/2 ph or pw
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(6, 8))

In [53]:
comp_conv2d(conv2d, X).shape

torch.Size([6, 8])

In [56]:
# 采用高度与宽度不同的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(1, 2))
X = torch.rand(size=(6, 8))

In [57]:
comp_conv2d(conv2d, X).shape

torch.Size([6, 8])

In [58]:
# 步幅
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
X = torch.rand(size=(8, 8))

In [59]:
comp_conv2d(conv2d, X).shape

torch.Size([4, 4])

In [60]:
# 步幅
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=(0, 1), stride=(2, 2))
X = torch.rand(size=(8, 8))

In [61]:
comp_conv2d(conv2d, X).shape

torch.Size([3, 4])